In [1]:
!pip install wandb
!pip install 'transformers[torch]'
!pip install datasets 

In [2]:
import wandb
import torch
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve

from transformers import AutoTokenizer, AutoModelForSequenceClassification, default_data_collator, XLMRobertaForSequenceClassification

from torch.utils.data import DataLoader, TensorDataset

from tqdm import tqdm

from IPython.display import clear_output
from datasets import load_dataset, Dataset

from sklearn.model_selection import train_test_split

2024-03-15 16:10:05.786352: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-15 16:10:05.786444: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-15 16:10:05.949789: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
def calc_metrics(y_true, y_pred):
    y_true = y_true.cpu()
    y_pred = y_pred.cpu()
    return accuracy_score(y_true, y_pred), precision_score(y_true, y_pred, zero_division=0), recall_score(y_true, y_pred, zero_division=0), f1_score(y_true, y_pred, zero_division=0)

@torch.no_grad()
def test(model, loader, device, tqdm_desc):
    loss_log = []
    acc_log = []
    prec_log = []
    rec_log = []
    f1_log = []
    model.eval()
    loss_func = nn.CrossEntropyLoss()

    for input_ids, attention_mask, labels in loader:
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        out = model(input_ids, attention_mask=attention_mask)
        loss = loss_func(out.logits, labels)

        loss_log.append(loss.item())

        pred = torch.argmax(out.logits, dim=1)
        res = calc_metrics(labels, pred)
        acc_log.append(res[0])
        prec_log.append(res[1])
        rec_log.append(res[2])
        f1_log.append(res[3])

    return loss_log, acc_log, prec_log, rec_log, f1_log


def train(model, optimizer, n_epochs, train_loader, val_loader, batch_size, scheduler=None):
    train_loss = []
    train_acc = []
    train_prec = []
    train_rec = []
    train_f1 = []
    
    run = wandb.init(project='html classificator', reinit=True)
    wandb.watch(model, nn.CrossEntropyLoss(), log="all", log_freq=100)
    model.train()

    batch = 0
    loss_func = nn.CrossEntropyLoss()
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    for epoch in range(n_epochs):
        for input_ids, attention_mask, labels in tqdm(train_loader, desc=f'Training {epoch}/{n_epochs}'):
            batch += 1
            
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            out = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = loss_func(out.logits, labels)
            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

            pred = torch.argmax(out.logits, dim=1)
            res = calc_metrics(labels, pred)
            train_acc.append(res[0])
            train_prec.append(res[1])
            train_rec.append(res[2])
            train_f1.append(res[3])
            
            if batch == 500:
                batch = 0
                
                val_loss, val_acc, val_prec, val_rec, val_f1 = test(model, val_loader, device, tqdm_desc='Validating')
                model.train()
                
                wandb.log({"train": {"acc": np.mean(train_acc), "pre": np.mean(train_prec), "rec": np.mean(train_rec), "f1": np.mean(train_f1), "loss": np.mean(train_loss)}, 
                           "val": {"acc": np.mean(val_acc), "pre": np.mean(val_prec), "rec": np.mean(val_rec), "f1": np.mean(val_f1), "loss": np.mean(val_loss)}})
                
                # clear_output()
                print(f"Next 100 batches:")
                print(f" train loss: {np.mean(train_loss)}, train acc: {np.mean(train_acc)}")
                print(f" val loss: {np.mean(val_loss)}, val acc: {np.mean(val_acc)}\n")
                train_loss = []
                train_acc = []
                train_prec = []
                train_rec = []
                train_f1 = []

        if scheduler is not None:
            scheduler.step()
            
    # последние батчи
    val_loss, val_acc, val_prec, val_rec, val_f1 = test(model, val_loader, device, tqdm_desc='Validating')
                
    wandb.log({"train": {"acc": np.mean(train_acc), "pre": np.mean(train_prec), "rec": np.mean(train_rec), "f1": np.mean(train_f1), "loss": np.mean(train_loss)}, 
               "val": {"acc": np.mean(val_acc), "pre": np.mean(val_prec), "rec": np.mean(val_rec), "f1": np.mean(val_f1), "loss": np.mean(val_loss)}})
    print(f"Last batches:")
    print(f" train loss: {np.mean(train_loss)}, train acc: {np.mean(train_acc)}")
    print(f" val loss: {np.mean(val_loss)}, val acc: {np.mean(val_acc)}\n")

    wandb.unwatch()
    run.finish()

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [6]:
# Load model directly

tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")
model = XLMRobertaForSequenceClassification.from_pretrained("FacebookAI/xlm-roberta-base").to(device)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
table_path = "/kaggle/input/site-bin-class/pars_data.csv"
data = pd.read_csv(table_path, on_bad_lines='skip', sep=';', lineterminator='\n')
data.dropna(axis=0, how='any', inplace=True)
data.text = data.text.astype(str)

dataset = Dataset.from_pandas(data)

In [7]:
data.head()

,text,target
0,индивидуальные групповые занятия будни выходны...,1
1,royal life федеральное театральное агентство «...,0
2,автоматизированная система uontravel войти заб...,0
3,уходовая домашняя косметика для души тела зада...,0
4,новости обучение трейдингу нуля обучение трейд...,1


In [8]:
print('общий размер датасета', data.shape[0])
print('количество элементов не из образования', data[data['target'] == 0]['text'].count())
print('количество элементов из образования', data[data['target'] == 1]['text'].count())
print('максимальный размер строки в датасете', max(len(i) for i in data['text']))

общий размер датасета 47621
количество элементов не из образования 34656
количество элементов из образования 12965
максимальный размер строки в датасете 1339641


In [8]:
TEST_SIZE = 0.3
SPLIT_RANDOM_SEED = 42
MAX_LENGTH = 1000000

def encode(examples):
    result = tokenizer(examples["text"], truncation=True, max_length=512, padding="max_length")
    return result

tokenized_datasets = dataset.map(encode, batched=True, remove_columns="text")

  0%|          | 0/48 [00:00<?, ?ba/s]

In [9]:
input_ids_train, input_ids_v, attention_mask_train, attention_mask_v, label_train, label_v = train_test_split(torch.tensor(tokenized_datasets['input_ids']), 
                                                                                                  torch.tensor(tokenized_datasets['attention_mask']), 
                                                                                                  torch.tensor(tokenized_datasets['target']), 
                                                                                                  random_state=SPLIT_RANDOM_SEED, test_size=TEST_SIZE, shuffle=True)

input_ids_val, input_ids_test, attention_mask_val, attention_mask_test, label_val, label_test = train_test_split(input_ids_v, attention_mask_v, label_v, 
                                                                                                  random_state=SPLIT_RANDOM_SEED, test_size=0.33, shuffle=True)

train_dataset = TensorDataset(input_ids_train, attention_mask_train, label_train)
val_dataset = TensorDataset(input_ids_val, attention_mask_val, label_val)
test_dataset = TensorDataset(input_ids_test, attention_mask_test, label_test)

In [10]:
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [11]:
wandb.login(key="eba16103be2afd0b5c96243771d60f5d7e562f68")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [14]:
# показания модели imdb (преобученная)
# model = model.to(device)
val_loss, val_acc, val_prec, val_rec, val_f1 = test(model, val_loader, device, tqdm_desc='Test')
print(f'Без обучения на датасете (чисто модель с параметрами) loss = {np.mean(val_loss)}, accuracy = {np.mean(val_acc)}, precision = {np.mean(val_prec)}, recall = {np.mean(val_rec)}, f1_score = {np.mean(val_f1)}')

Test: 100%|██████████| 599/599 [05:07<00:00,  1.95it/s]

Без обучения на датасете (чисто модель с параметрами) loss = 0.8441950186266127, accuracy = 0.27013772954924875, precision = 0.27013772954924875, recall = 0.991652754590985, f1_score = 0.4131570013260478


In [24]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [12]:
optimizer = optim.Adam(model.parameters(), lr=2e-5, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=3, eta_min=3e-10)
train(model, optimizer, 3, train_loader, val_loader, batch_size, scheduler)

wandb: Currently logged in as: rodion-chernomordin. Use `wandb login --relogin` to force relogin


Training 0/3:  24%|██▍       | 500/2084 [19:33<47:37:06, 108.22s/it]

Next 100 batches:
 train loss: 0.3438436353430152, train acc: 0.84425
 val loss: 0.25169663431623546, val acc: 0.8907554257095158



Training 0/3:  48%|████▊     | 1000/2084 [39:11<32:32:22, 108.07s/it]

Next 100 batches:
 train loss: 0.2522452085465193, train acc: 0.8975
 val loss: 0.25980170067082564, val acc: 0.8956594323873122



...

Next 100 batches:
 train loss: 0.24453319256380202, train acc: 0.89575
 val loss: 0.23990811820364308, val acc: 0.8995200333889817



Training 2/3:  88%|████████▊ | 1832/2084 [1:13:57<7:34:22, 108.18s/it]

Next 100 batches:
 train loss: 0.24176935820281506, train acc: 0.897375
 val loss: 0.2338708041232695, val acc: 0.9017111853088481



Training 2/3: 100%|██████████| 2084/2084 [1:20:55<00:00,  2.33s/it]   


Last batches:
 train loss: 0.24218072983184977, train acc: 0.8946759259259259
 val loss: 0.23673500844241682, val acc: 0.9010851419031719



In [13]:
test_loss, test_acc, test_prec, test_rec, test_f1 = test(model, test_loader, device, tqdm_desc='Test')
print(f'loss = {np.mean(test_loss)}, accuracy = {np.mean(test_acc)}, precision = {np.mean(test_prec)}, recall = {np.mean(test_rec)}, f1_score = {np.mean(test_f1)}')

loss = 0.22772546339211827, accuracy = 0.9044491525423729, precision = 0.8426836158192089, recall = 0.7995466774280334, f1_score = 0.798695709903665
